In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [9]:
rating = pd.read_csv('ratings.csv')
architect = pd.read_csv('architects.csv')
user = pd.read_csv('users.csv');
architect_rating = pd.merge(rating, architect, on='architect_id')
cols = ['Name', 'Address 1', 'Country']
architect_rating.drop(cols, axis=1, inplace=True)

In [17]:
rating_count = (architect_rating,
                groupby(by = [''])
               )

,Registration,Name,Company,WorkPhone,Address 1,Address 2,Address 3,City,State,Postcode,Country,Member Type
0,499,Mr Brian Hodgen,Hodgen & Hall Architects,(07) 4639-1566,L1/ 453 Ruthven Street,NaN,NaN,TOOWOOMBA,QLD,4350,NaN,ARCH
1,720,Mr Desmond Donaldson,Des Donaldson Architect,412199260,42 Kalmia Drive,NaN,NaN,Redland Bay,QLD,4165,NaN,ARCH
2,737,Mr Ronald Baker,Ronald G Baker Architect,07 3397 3207,15 Verry Street,NaN,NaN,COORPAROO,QLD,4151,NaN,ARCH
3,795,Mr Kenneth O'Shea,Ken O'Shea Architect,(07) 5533-5174,284 Lyre-Bird Ridge Road Springbrook,NaN,NaN,SPRINGBROOK,QLD,4213,NaN,ARCH
4,809,Mr John Walsh,John Walsh Architects Pty Ltd,(07) 3252-0208,16 Upper Lancaster Road,NaN,NaN,ASCOT,QLD,4007,NaN,ARCH


In [18]:
print(ratings_df.shape)
print(ratings_df.user_id.nunique())
print(ratings_df.architect_id.nunique())
ratings_df.isna().sum()

(2000, 3)
100
875


user_id         0
architect_id    0
rating          0
dtype: int64

In [31]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest = train_test_split(ratings_df, test_size=0.01, random_state=1)
print(f"Shape of train data: {Xtrain.shape}")
print(f"Shape of test data: {Xtest.shape}")

Shape of train data: (1980, 3)
Shape of test data: (20, 3)


In [32]:
#Get the number of unique entities in architects and users columns
narchitect_id = ratings_df.architect_id.nunique()
nuser_id = ratings_df.user_id.nunique()

In [33]:
#Architect input network
input_architects = tf.layers.Input(shape=(1,))
embed_architects = tf.layers.Embedding(narchitect_id + 1, 15)(input_architects)
architects_out = tf.layers.Flatten()(embed_architects)

#User input network
input_users = tf.layers.Input(shape=(1,))
embed_users = tf.layers.Embedding(nuser_id + 1, 15)(input_users)
users_out = tf.layers.Flatten()(embed_users)

conc_layer = tf.layers.Concatenate()([architects_out, users_out])
x = tf.layers.Dense(128, activation='relu')(conc_layer)
x_out = x = tf.layers.Dense(1, activation='relu')(x)
model = tf.Model([input_architects, input_users], x_out)

In [34]:
#Compile network
opt = tf.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')

model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 15)        13140       input_10[0][0]                   
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 1, 15)        1515        input_11[0][0]                   
_______________________________________________________________________________________

In [35]:
hist = model.fit(
    [Xtrain.architect_id, Xtrain.user_id], Xtrain.rating, 
    batch_size=16, 
    epochs=2, 
    verbose=1,
    validation_data=([Xtest.architect_id, Xtest.user_id], Xtest.rating)
)

Epoch 1/2


InvalidArgumentError:  indices[5,0] = 987 is not in [0, 876)
	 [[node functional_9/embedding_8/embedding_lookup (defined at <ipython-input-35-2c61efdf5b9c>:1) ]] [Op:__inference_train_function_3845]

Errors may have originated from an input operation.
Input Source operations connected to node functional_9/embedding_8/embedding_lookup:
 functional_9/embedding_8/embedding_lookup/3596 (defined at /usr/lib/python3.8/contextlib.py:113)

Function call stack:
train_function
